In [1]:
#Full space model
from pyomo.environ import *
from pyomo.opt import SolverFactory
import itertools

import pandas as pd

model = ConcreteModel()

# 5D Array
model.gen = Set(initialize=[1, 2])  # Generators
model.Periods = Set(initialize=[1, 2, 3])  # Time periods
model.days = Set(initialize=[1, 2, 3])  # Demand scenario for time period 1

# Total Scenarios

file_path = 'newgenexp.xlsx'
sheet_name = 'DIC'  # Specify the sheet name or sheet number

# Load the data into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
c = {}
for i in range(0,df.shape[0]):
    c[df['Generator'][i]] = df['Ammortized Fixed operating cost($)'][i]
model.c = Param(model.gen, initialize=c)  # Fixed cost per unit capacity of generator j

sheet_name = 'FIXOP'  # Specify the sheet name or sheet number

# Load the data into a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
f = {}
for i in range(0,df.shape[0]):
    f[(df['Generator'][i],df['Period'][i])] = df['FIXOP'][i]
model.f = Param(model.gen,model.Periods, initialize=f)  # Fixed cost per unit capacity of generator j

sheet_name = 'Additional cost'  # Specify the sheet name or sheet number

df = pd.read_excel(file_path, sheet_name=sheet_name)
g = {}
for i in range(0,df.shape[0]):
    g[df['Day'][i]] = df['Cost'][i]
model.g = Param(model.days, initialize=g)



d = {}

sheet_name = 'Demand'
df = pd.read_excel(file_path, sheet_name=sheet_name)
for i in range(0,df.shape[0]):
    d[df['Period'][i],df['Day'][i]]= df['Demand (KWh)'][i]
    
                       
               
model.d = Param(model.Periods,model.days, initialize=d)  # Fixed cost per unit capacity of generator j

                       
   
a = {}    
sheet_name = 'Availability'
df = pd.read_excel(file_path, sheet_name=sheet_name)
for i in range(0,df.shape[0]):
    a[df['Period'][i],df['Day'][i],df['Generator'][i]]= df['Availibility'][i]
    
                       
model.a = Param(model.Periods,model.days,model.gen, initialize=a)  # Fixed cost per unit capacity of generator j

# Variables
model.x = Var(model.gen, domain=NonNegativeReals)  # Installed capacity of generator j
model.y = Var(model.Periods, model.days, model.gen, domain=NonNegativeReals)  # Operating level of generator j
model.y_purchased = Var(model.Periods, model.days, domain=NonNegativeReals)  # Additional capacity purchased
# Objective function
def objective_rule(model):
    fixed_cost = 3*sum(model.c[j] * model.x[j] for j in model.gen)
    operating_cost = sum(sum(sum(model.f[j,i] * model.y[i,k, j] for j in model.gen) + model.g[k] * model.y_purchased[i,k] for i in model.Periods) for k in model.days)
    return fixed_cost + operating_cost

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints

# Demand satisfaction constraints
def demand_satisfaction_rule(model, i, k):
    return sum(model.y[i,k,j] for j in model.gen) + model.y_purchased[i,k] >= model.d[i,k]
model.demand_satisfaction_constraint = Constraint(model.Periods, model.days, rule=demand_satisfaction_rule)

# Availability constraints
def availability_rule(model, i, k, gen):
    return model.y[i,k,gen] <= model.a[i,k,gen] * model.x[gen]
model.availability_constraint = Constraint(model.Periods, model.days, model.gen, rule=availability_rule)

opt = SolverFactory('gurobi')
results = opt.solve(model, tee=True)

model.pprint()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmpag1f4k0m.pyomo.lp
Reading time = 0.00 seconds
x1: 27 rows, 29 columns, 63 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 27 rows, 29 columns and 63 nonzeros
Model fingerprint: 0x6b3cb3e3
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [5e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 2e+03]
Presolve time: 0.01s
Presolved: 27 rows, 29 columns, 63 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.533327e+03   0.000000e+00      0s
      20    3.5740898e+05   0.000000e+00   0.000000e+00